- adicionar coluna valor de venda (+1)
- adicionar nota ENEM (+2)
- adicionar coords (+2 -1)
- adiciona se a escola já é do grupo (+1)

In [1]:
import pandas as pd
import json
from utils.busca_ceps import cep_to_coords


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

In [2]:
df_edb_raw = pd.read_csv("dados/temporarios/01-md_edb_tratado.csv",
                 dtype={"CO_CEP": str})

df_edb_raw.info()  # 37_178 linhas, 23 colunas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37178 entries, 0 to 37177
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CO_ENTIDADE          37178 non-null  int64  
 1   CO_CEP               37178 non-null  object 
 2   QT_SALAS_UTILIZADAS  37178 non-null  float64
 3   IN_EXAME_SELECAO     37178 non-null  float64
 4   QT_MAT_INF           37178 non-null  float64
 5   QT_MAT_FUND_AI       37178 non-null  float64
 6   QT_MAT_FUND_AF       37178 non-null  float64
 7   QT_MAT_MED           37178 non-null  float64
 8   infraestrutura       37178 non-null  int64  
 9   estrutura_pobre      37178 non-null  int64  
 10  estrutura_basica     37178 non-null  int64  
 11  estrutura_padrao     37178 non-null  int64  
 12  estrutura_premium    37178 non-null  int64  
 13  qt_ativos_basico     37178 non-null  int64  
 14  qt_ativos_premium    37178 non-null  int64  
 15  pessoal_basico       37178 non-null 

## Valor de Venda

In [3]:
df = df_edb_raw.copy()

with open("dados/inputs/ticket_medio.json", "r", encoding="utf-8") as f:
    ticket_medio = json.load(f)


df["valor_venda"] = (
    df["QT_MAT_INF"] * ticket_medio["ei"]
    + df["QT_MAT_FUND_AI"] * ticket_medio["efai"]
    + df["QT_MAT_FUND_AF"] * ticket_medio["efaf"]
    + df["QT_MAT_MED"] * ticket_medio["em"]
    )


print(df.shape)
display(df.head())


(37178, 24)


,CO_ENTIDADE,CO_CEP,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao,alunos_p_professor,alunos_p_sala,valor_venda
0,11006765,76870450,12.0,0.0,113.0,236.0,148.0,0.0,2,0,2,2,2,19,1,7,2,0,497,32,3.0,15.531250,41.416667,567134.87
1,11007028,76870692,7.0,0.0,309.0,0.0,0.0,0.0,2,0,1,1,1,3,2,7,2,0,309,9,3.0,34.333333,44.142857,173815.59
2,11007656,76870505,25.0,0.0,113.0,216.0,158.0,158.0,2,0,4,4,4,40,8,16,8,2,645,44,3.0,14.659091,25.800000,888919.23
3,11044454,76872856,7.0,0.0,308.0,0.0,0.0,0.0,2,1,3,2,2,8,0,9,22,2,308,9,3.0,34.222222,44.000000,173253.08
4,11044462,76874166,27.0,0.0,178.0,343.0,285.0,0.0,2,0,3,2,1,20,0,11,5,5,806,40,3.0,20.150000,29.851852,953800.11


## Nota Enem

In [4]:
df_escolas_enem = pd.read_csv("dados/temporarios/01-escolas_enem.csv")

df_escolas_enem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26302 entries, 0 to 26301
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   cod_escola  26302 non-null  int64  
 1   nota_enem   26302 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 411.1 KB


In [5]:
df = df.merge(df_escolas_enem, how="left", left_on="CO_ENTIDADE", right_on="cod_escola")

df = df.drop(columns=["cod_escola"])

display(df.head())
print(df.shape)


,CO_ENTIDADE,CO_CEP,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao,alunos_p_professor,alunos_p_sala,valor_venda,nota_enem
0,11006765,76870450,12.0,0.0,113.0,236.0,148.0,0.0,2,0,2,2,2,19,1,7,2,0,497,32,3.0,15.531250,41.416667,567134.87,NaN
1,11007028,76870692,7.0,0.0,309.0,0.0,0.0,0.0,2,0,1,1,1,3,2,7,2,0,309,9,3.0,34.333333,44.142857,173815.59,NaN
2,11007656,76870505,25.0,0.0,113.0,216.0,158.0,158.0,2,0,4,4,4,40,8,16,8,2,645,44,3.0,14.659091,25.800000,888919.23,1382.800735
3,11044454,76872856,7.0,0.0,308.0,0.0,0.0,0.0,2,1,3,2,2,8,0,9,22,2,308,9,3.0,34.222222,44.000000,173253.08,NaN
4,11044462,76874166,27.0,0.0,178.0,343.0,285.0,0.0,2,0,3,2,1,20,0,11,5,5,806,40,3.0,20.150000,29.851852,953800.11,NaN


(37178, 25)


## Coordenadas

In [6]:
df = await cep_to_coords(df, "CO_CEP")


Temos 28935 CEPs diferentes a consultar
Nao foi necessario baixar novas coods


In [7]:
print(df.shape)
display(df.head())


(37178, 26)


,CO_ENTIDADE,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao,alunos_p_professor,alunos_p_sala,valor_venda,nota_enem,lat,lon
0,11006765,12.0,0.0,113.0,236.0,148.0,0.0,2,0,2,2,2,19,1,7,2,0,497,32,3.0,15.531250,41.416667,567134.87,NaN,-9.91131,-63.04099
1,11007028,7.0,0.0,309.0,0.0,0.0,0.0,2,0,1,1,1,3,2,7,2,0,309,9,3.0,34.333333,44.142857,173815.59,NaN,-9.89796,-63.04024
2,11007656,25.0,0.0,113.0,216.0,158.0,158.0,2,0,4,4,4,40,8,16,8,2,645,44,3.0,14.659091,25.800000,888919.23,1382.800735,-9.90869,-63.03561
3,11044454,7.0,0.0,308.0,0.0,0.0,0.0,2,1,3,2,2,8,0,9,22,2,308,9,3.0,34.222222,44.000000,173253.08,NaN,-9.90840,-63.03315
4,11044462,27.0,0.0,178.0,343.0,285.0,0.0,2,0,3,2,1,20,0,11,5,5,806,40,3.0,20.150000,29.851852,953800.11,NaN,-9.89198,-63.03205


## Se já é Cliente

In [8]:
df_cliente = pd.read_excel("dados/inputs/escolas_atuais.xlsx")


df_cliente["cliente"] = 1

df_cliente = df_cliente.melt(id_vars="cliente",
             value_vars=["Código INEP 1", "Código INEP 2", "Código INEP 3"],
             value_name="co_inep")

df_cliente = (df_cliente.dropna(subset="co_inep").drop(columns="variable")
              .drop_duplicates())

df_cliente["co_inep"] = df_cliente["co_inep"].astype(int)

print(df_cliente.shape)
display(df_cliente.head())


(556, 2)


,cliente,co_inep
0,1,35155226
1,1,35169511
2,1,35004821
3,1,26156407
4,1,35008207


In [9]:
df = df.merge(df_cliente, how="left", left_on="CO_ENTIDADE", right_on="co_inep")
df = df.drop(columns="co_inep")

print(df.value_counts("cliente", dropna=False))  # 533 matches de 556 escolas atuais (perdemos algumas)


cliente
NaN    36645
1.0      533
Name: count, dtype: int64


## Salvar

In [10]:
display(df.head())
print(df.shape)


,CO_ENTIDADE,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao,alunos_p_professor,alunos_p_sala,valor_venda,nota_enem,lat,lon,cliente
0,11006765,12.0,0.0,113.0,236.0,148.0,0.0,2,0,2,2,2,19,1,7,2,0,497,32,3.0,15.531250,41.416667,567134.87,NaN,-9.91131,-63.04099,NaN
1,11007028,7.0,0.0,309.0,0.0,0.0,0.0,2,0,1,1,1,3,2,7,2,0,309,9,3.0,34.333333,44.142857,173815.59,NaN,-9.89796,-63.04024,NaN
2,11007656,25.0,0.0,113.0,216.0,158.0,158.0,2,0,4,4,4,40,8,16,8,2,645,44,3.0,14.659091,25.800000,888919.23,1382.800735,-9.90869,-63.03561,1.0
3,11044454,7.0,0.0,308.0,0.0,0.0,0.0,2,1,3,2,2,8,0,9,22,2,308,9,3.0,34.222222,44.000000,173253.08,NaN,-9.90840,-63.03315,NaN
4,11044462,27.0,0.0,178.0,343.0,285.0,0.0,2,0,3,2,1,20,0,11,5,5,806,40,3.0,20.150000,29.851852,953800.11,NaN,-9.89198,-63.03205,NaN


(37178, 27)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37178 entries, 0 to 37177
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CO_ENTIDADE          37178 non-null  int64  
 1   QT_SALAS_UTILIZADAS  37178 non-null  float64
 2   IN_EXAME_SELECAO     37178 non-null  float64
 3   QT_MAT_INF           37178 non-null  float64
 4   QT_MAT_FUND_AI       37178 non-null  float64
 5   QT_MAT_FUND_AF       37178 non-null  float64
 6   QT_MAT_MED           37178 non-null  float64
 7   infraestrutura       37178 non-null  int64  
 8   estrutura_pobre      37178 non-null  int64  
 9   estrutura_basica     37178 non-null  int64  
 10  estrutura_padrao     37178 non-null  int64  
 11  estrutura_premium    37178 non-null  int64  
 12  qt_ativos_basico     37178 non-null  int64  
 13  qt_ativos_premium    37178 non-null  int64  
 14  pessoal_basico       37178 non-null  int64  
 15  pessoal_padrao       37178 non-null 

In [12]:
df.to_csv("dados/temporarios/02-dados_escolas.csv", index=False)
